In [1]:
SEASON = '2018-19'

In [2]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,James Harden,HOU,PG,19.706339,29,78,78,36.8,10.8,24.5,...,40.5,11.4,3.8,15.2,0.254,9.4,1.6,11.0,9.3,53.0
2,Giannis Antetokounmpo,MIL,PF,17.974481,24,72,72,32.8,10.0,17.3,...,32.3,8.9,5.5,14.4,0.292,6.3,4.1,10.4,7.4,60.0
3,Nikola Jokić,DEN,C,14.416143,23,80,80,31.3,7.7,15.1,...,27.4,7.5,4.3,11.8,0.226,6.2,2.9,9.1,7.0,54.0
4,Paul George,OKC,SF,13.487288,28,77,77,36.9,9.2,21.0,...,29.5,7.0,4.9,11.9,0.201,5.6,1.6,7.2,6.6,49.0
5,Damian Lillard,POR,PG,12.583206,28,80,80,35.5,8.5,19.2,...,29.3,9.7,2.4,12.1,0.205,6.6,-0.3,6.4,6.0,53.0
6,Kevin Durant,GSW,SF,12.040905,30,78,78,34.6,9.2,17.7,...,29.0,8.6,2.9,11.5,0.204,5.4,0.1,5.5,5.1,57.0
7,Stephen Curry,GSW,PG,11.705505,30,69,69,33.8,9.2,19.4,...,30.4,7.2,2.5,9.7,0.199,7.1,-0.5,6.6,5.1,57.0
8,Kawhi Leonard,TOR,SF,11.667303,27,60,60,34.0,9.3,18.8,...,30.3,6.1,3.4,9.5,0.224,6.1,1.1,7.2,4.7,58.0
9,Anthony Davis,NOP,C,11.637840,25,56,56,33.0,9.5,18.3,...,29.5,6.4,3.1,9.5,0.247,7.1,2.3,9.4,5.3,33.0
10,Rudy Gobert,UTA,C,11.207123,26,81,80,31.8,5.9,8.8,...,17.8,8.7,5.7,14.4,0.268,3.0,2.5,5.4,4.8,50.0
